<a href="https://colab.research.google.com/github/gsk224/Spatial_Portfolio/blob/main/CambodiaSochet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering Illegal Cambodian Gold Mines Using Sentinal 1 Change Detection Algorythm

# Ingest

In [1]:
import ee

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project="ee-gk-mapmatics")

In [4]:
import geemap
import folium
import pandas as pd
import geopandas as gpd
import os

In [8]:
!pip install PyGithub
from github import Github

# Sentinel 2 Maps with change data

**Split Map with Sentinel 2 Images**

Access Sentinel 2 tiles that overlay the study area which show the forest being cleared for access roads and mines opening. The image collection were filtered to only incorperate ones showing the study area, with a cloud cover of less than %5, and within the dates of pre and post event.

The filtered images were then explored one by one for accuracy and selected by highest visibility of forest cover and change.

In [6]:
#  SET Area of Interest

aoipolygon = ee.Geometry.Polygon([
          [[105.762634, 12.952033],
           [105.762634, 13.015931],
           [105.849495, 13.015931],
           [105.849495, 12.952033],
           [105.762634, 12.952033]]
          ], None, False)

aoi = ee.FeatureCollection([aoipolygon])

# Create bounding box image

empty = ee.Image().byte()
aoi_outline = empty.paint(featureCollection=aoi, color=1, width=2)

In [39]:
# Images for split map, post event
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

imagesPost = (
    collection.filterBounds(aoi)
    .filterDate("2023-08-01", "2024-08-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)
imagesPost.size()

In [40]:
# Images for splitmap, pre event

imagesPre = (
    collection.filterBounds(aoi)
    .filterDate("2021-06-01", "2021-11-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)
imagesPre.size()

In [41]:
# Selecting images for splitmap

image0322 = imagesPre
image1223 = ee.Image(imagesPost.toList(13).get(0))

In [42]:
# Visualising the Sentinel 2 images
visS2 = {
    "min": 0.0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],
}

In [43]:
# Split map displaying land use change over 18 months
m = geemap.Map()
m.centerObject(aoi, 12)

right_layer = geemap.ee_tile_layer(image1223, visS2 , 'December 2023')
left_layer = geemap.ee_tile_layer(image0322, visS2 , 'March 2022')
m.split_map(left_layer, right_layer)

m

# In the centre of the map it clearly shows the carving of access roads into the forest, and opening of what look like mines.
# To the North West of the split map, it shows the expansion of existing gold and copper mines.

Map(center=[12.983981834349013, 105.80606449999864], controls=(ZoomControl(options=['position', 'zoom_in_text'…

**TimeLapse**
will show key dates of changes

In [ ]:
# Annual change over the AOI

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2018,
    end_year=2025,
    start_date="01-01",
    end_date="12-31",
    frequency="year",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

In [ ]:
# Monthly change from start2022 - end2023

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2022,
    end_year=2023,
    start_date="01-01",
    end_date="12-31",
    frequency="month",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

In [ ]:
# Monthly change from start2023 - end2024

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2023,
    end_year=2024,
    start_date="01-01",
    end_date="12-31",
    frequency="month",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

**Geopandas**
source data, clean it, and filter down. Can do analysis on concession areas.

Import change detection analysis results from GEE assets

In [45]:
# Gold Mining Alerts Filtered 2022

gm22 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2022')

# Gold Mining Alerts Filtered 2023

gm23 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2023')

# Gold Mining Alerts Filtered 2023

gm24 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2024')


In [46]:
# Change Map Filtered 2022

cm22 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2022')

# Change Map Filtered 2023

cm23 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2023')

# Change Map Filtered 2023

cm24 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2024')

In [49]:
# Change Map Visualisation

jet = ['black', 'blue', 'cyan', 'yellow', 'red']
vis_gee = {
    'min': 0,
    'max': 30,
    'palette': jet
}

Next visualise the known mining concessions granted in the area and the boundaries of the Prey Lang Wildlife Sanctuary. Shows that the new mines fall outside of known concessions, and deep within the wildlife sanctuary boarders.

In [13]:
# Select Mining Cocessions Layer
url_m = "https://github.com/gsk224/Spatial_Portfolio/raw/main/miningconcessions.gpkg"
mining_gdf = gpd.read_file(url_m)
mining_gdf.head()

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on '/vsicurl/https://github.com/gsk224/Spatial_Portfolio/raw/main/miningconcessions.gpkg'
  return ogr_read(


,map_id,min_name,in_country,address,dir_name,dir_nation,mining_reg,com_web,con_date,issu_minis,...,district,commune,data_class,reference,referen_kh,last_updat,language,cap_inv_m,cap_inv,geometry
0,401,"CONTINENTAL COPPER (CAMBODIA) CO., LTD.",Cambodia,"I23-27, Khmounh Village, Sangkat Khmounh, Khan...",James Andrew STEPHEN,Not found,Not found,Not found,Not found,Not found,...,Not found,Khvav; Sre Yang Ronakse,Government data complete,Notification_no_001__10.01.2020.pdf;announceme...,,3/30/2020,English,NaN,NaN,"MULTIPOLYGON (((104.47699 13.5696, 104.50964 1..."
1,402,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,...,Thala Barivat,Not found,Government data complete,Notification_no_001__10.01.2020.pdf,,1/14/2020,English,NaN,NaN,"MULTIPOLYGON (((105.64378 13.8407, 105.79181 1..."
2,403,Renaissance Minerals (Cambodia) Limited,Cambodia,"#26, St.222, Village 2, Sangkat Boeng Reang, K...",Bernard Joseph CLEARY,Not found,Not found,www.emeraldresources.com.au,Not found,Not found,...,Keo Seima,Not found,Government data complete,Notification_no_001__10.01.2020.pdf;announceme...,,3/30/2020,English,NaN,NaN,"MULTIPOLYGON (((106.89731 12.6332, 106.92952 1..."
3,404,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,...,Chaeb; Chey Sen,Not found,Government data complete,Notification_no_001__10.01.2020.pdf,,1/14/2020,English,NaN,NaN,"MULTIPOLYGON (((105.36594 13.71565, 105.47708 ..."
4,405,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,Not found,...,Chey Sen,Not found,Government data complete,Notification_no_001__10.01.2020.pdf,,1/14/2020,English,NaN,NaN,"MULTIPOLYGON (((105.30103 13.51587, 105.32968 ..."


In [9]:
# Select Protected Areas Layer
url_pa = "https://github.com/gsk224/Spatial_Portfolio/raw/c4635887d532c4765a5fefdbc509005a040614c7/wildlife.gpkg"
pa_gdf = gpd.read_file(url_pa)
pa_gdf.head()

/usr/local/lib/python3.11/dist-packages/pyogrio/raw.py:198: RuntimeWarning: GPKG: unrecognized user_version=0x00000000 (0) on '/vsicurl/https://github.com/gsk224/Spatial_Portfolio/raw/c4635887d532c4765a5fefdbc509005a040614c7/wildlife.gpkg'
  return ogr_read(


,ogc_fid0,map_id,name,size_ha,province,district,commune,type_area,category,categor_en,issuedate,reference,referen_kh,last_update,language,geometry
0,1,pa_ws_74,Prey Lang Wildlife Sanctuary,"431,683","Stung Treng, Kampong Thom, Kratie, Preah Vihear",Not found,Not found,Natural Protected Areas,Wildlife Sanctuary,Wildlife Sanctuary,9/5/2016,Sub_decree_No_74__09.05.2016.pdf,None,25/5/2016,English,"MULTIPOLYGON (((556152.767 1519643.916, 556681..."
1,40,pa_ws_41,Snuol Wildlife Sanctuary,"75,000",Kratie,Not found,Not found,Natural Protected Areas,Wildlife Sanctuary,Wildlife Sanctuary,1/11/1993,Royal_Decree_on_Protected_Areas__01.11.1993.pdf,None,22/5/2014,English,"MULTIPOLYGON (((686923.501 1352318.103, 687121..."
2,3,pa_ws_75,Preah Rokar Wildlife Sanctuary,"90,361",Preah Vihear,Not found,Not found,Natural Protected Areas,Wildlife Sanctuary,Wildlife Sanctuary,9/5/2016,Sub_decree_No_75__09.05.2016.pdf,None,25/5/2016,English,"MULTIPOLYGON (((515782.121 1558284.185, 515914..."
3,4,pa_ws_2,Phnom Prich Wildlife Sanctuary,"222,500","Ratanak Kiri, Mondul Kiri",Not found,Not found,Natural Protected Areas,Wildlife Sanctuary,Wildlife Sanctuary,1/11/1993,Royal_Decree_on_Protected_Areas__01.11.1993.pdf,None,22/5/2014,English,"MULTIPOLYGON (((715065.838 1433459.425, 715124..."
4,5,pa_ws_3,Lomphat Wildlife Sanctuary,"250,000","Ratanak Kiri, Mondul Kiri",Not found,Not found,Natural Protected Areas,Wildlife Sanctuary,Wildlife Sanctuary,1/11/1993,Royal_Decree_on_Protected_Areas__01.11.1993.pdf,None,22/5/2014,English,"MULTIPOLYGON (((744784.677 1497527.388, 745432..."


In [16]:
# Check projections of both data sets
print(mining_gdf.crs)
print(pa_gdf.crs)

EPSG:4326
EPSG:32648


In [17]:
# Standardise to Geographic system 4326
pa_4326 = pa_gdf.to_crs(epsg=4326)

In [19]:
# Check. This makes sure they align on the map
print(mining_gdf.crs)
print(pa_4326.crs)

EPSG:4326
EPSG:4326


In [26]:
# Convert the reprojectded protected areas data to a FeatureCollection and filter for Prey Lang only
preylang_fc = geemap.gdf_to_ee(pa_4326).filter(ee.Filter.eq("name", "Prey Lang Wildlife Sanctuary"))

# Convert to a gdf for later processing
preylang_gdf = geemap.ee_to_gdf(preylang_fc)

In [35]:
# Only including concessions that intersect with Prey Lang boundary
mining_gdf_pl = mining_gdf[mining_gdf.geometry.intersects(preylang_gdf.geometry.union_all())]
mining_gdf_pl.shape


(31, 30)

In [20]:
# Prepare Visualisation for different mining concessions data sources. Filter by Clasification.

datasource = [
    'Government data complete',
    'Government data partial',
    'Secondary source data',
    'Other data',
]
colors = [
    '008000',
    'CC5500',
    'FAE033',
    '87CEEB',

]
fillColor = [c + "A8" for c in colors]

In [36]:
# Convert the concessions data to a FeatureCollection and styled with preset
mining_fc = geemap.gdf_to_ee(mining_gdf_pl)
concessions = ee.FeatureCollection(mining_fc).filter(
    ee.Filter.inList('data_class', datasource)
)
styled_concessions = geemap.ee_vector_style(concessions, column="data_class", labels=datasource, fillColor=fillColor, color='00000000')

In [117]:
# VISUALISE TIME
# Interactive map for exploring the data, highlighting aluvial mining activity with filtered change maps
# and alert maps that account for size of pool to eliminate false positives.
# Kept change maps because the alert filter requires tweeking as it removes too many results.
# Exploration of the average size of gold mine pool in the particular area is needed to refine results.
# Could be to do with prevalence of small scale 'artisinal' mining.

m = geemap.Map()
m.centerObject(aoi, 11)

m.addLayer(image1223, visS2, 'Dec 2023')

m.addLayer(styled_concessions, {}, 'Mining Concessions')
m.add_legend(title="Data Sources", labels=datasource, colors=colors)
m.add_layer(preylang_fc, {}, "Prey Lang Wildlife Sanctuary")

m.addLayer(aoi_outline, {'palette': 'red'}, 'Area of Study')

m.addLayer(gm22, vis_gee, 'Alerts Filtered 2022')
m.addLayer(gm23, vis_gee, 'Alerts Filtered 2023')
m.addLayer(gm24, vis_gee, 'Alerts Filtered 2024')
m.addLayer(cm22, vis_gee, 'Change Map Filtered 2022')
m.addLayer(cm23, vis_gee, 'Change Map Filtered 2023')
m.addLayer(cm24, vis_gee, 'Change Map Filtered 2024')
m


Map(center=[12.983981834348848, 105.80606449999864], controls=(WidgetControl(options=['position', 'transparent…

**Interpretation**

*   Map clearly shows huge areas of Prey Lang that have been sold to the mining industry by the cambodian government.
*   The "change maps filtered" layers show the detection of ground water potentially due to aluvial mining is increasing and spreading year on year in the study area.
*   The "Alerts Filtered" layers show an alert in 2024, with no alerts in the previous years. this could indicate increased activity in the mine, following the dramatic gold price rise in 2024

With these data sets i can now measure the area of prey lang, how many concessions lie within it, what area is taken up by mining concessions. and if the two new mines lie within a concession. I can also overlay forest loss data to show that the mining concessions correlate with the forest loss of the area.

In [108]:
# display forest loss data in the prey lang boundary.

hansen = ee.Image("UMD/hansen/global_forest_change_2023_v1_11")
clipped_hansen = hansen.clip(preylang_fc)
treeloss = clipped_hansen.select(["lossyear"])

treeLossVisParam = {"min": 0, "max": 23, "palette": ["blue", "red"]}

layer_name = "Tree loss 2000 - 2023"

In [69]:
# To work out % of Prey lang wildlife sanctuary mining concession cover will need to
# cut cocessions geometries to bondary of prey lang and input into data frame

mining_cut = mining_gdf_pl.copy()
mining_cut['geometry'] = mining_cut.geometry.intersection((preylang_gdf).geometry.iloc[0])

In [115]:
m = geemap.Map()
m.add_basemap("HYBRID")
m.center_object(preylang_fc, 9)

m.add_gdf(mining_cut, layer_name="Mining Concessions Cut", style={'color': 'yellow'})
m.addLayer(preylang_fc, {}, "Prey Lang Wildlife Sanctuary")
m.addLayer(aoi_outline, {'palette': 'red'}, 'Area of Study')
m.addLayer(treeloss, treeLossVisParam, layer_name)
m.add_colorbar(treeLossVisParam, label=layer_name, layer_name=layer_name)
m

Map(center=[13.240028202910644, 105.61153045085514], controls=(WidgetControl(options=['position', 'transparent…

In [86]:
preylang_ha = preylang_gdf["size_ha"].sum()
print(preylang_ha)

431,683


In [81]:
# reprojected the data to cambodian CRS to compute the area in hectares.
mining_cut = mining_cut.to_crs(epsg=32648)
mining_cut['size_m2'] = mining_cut.geometry.area
mining_cut['size_ha'] = mining_cut['size_m2']/10000
concessions_ha = mining_cut['size_ha'].sum()
print(concessions_ha)

251453.45634048275


In [87]:
preylang_ha = float(preylang_ha.replace(',', ''))
difference = preylang_ha - concessions_ha

percentage = (difference / preylang_ha) * 100

# results
print(f"Difference: {difference}")
print(f"Percentage Difference: {percentage}%")

Difference: 180229.54365951725
Percentage Difference: 41.750438089875495%


# Import, filter and visualise Sentinel 1 mosaics to create time series of the AOI
*   Adapted from work by Villa et al. (2024), Monitoring Gold Mining Activity
Using SAR, Cloud-Based Remote Sensing with Google Earth Engine
*   Could be done in GEE Javascript, however imorting the images into Python allowed more flexibility in using different packages to present data to people without access or knowledge of Earth Engine.




In [7]:
# Function to mask the SAR images acquired with an incidence angle
# lower or equal to 31 and greater or equal to 45 degrees.
def mask_angle(image):
    angle_mask = image.select('angle')
    return image.updateMask(angle_mask.gte(31).And(angle_mask.lte(45)))

# Function to get the SAR Collection.
def get_collection(dates, roi, orbit_pass):
    sar_coll_float = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
        .filterBounds(roi) \
        .filterDate(dates[0], dates[1]) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
    return sar_coll_float.map(mask_angle).select(['VV', 'VH'])

# Define variables: the period of time and the orbit pass.
list_of_dates = ['2022-03-01', '2022-08-01']
orbit_pass = 'DESCENDING'

# Apply the function to get the SAR collection.
sar_image_coll = get_collection(list_of_dates, aoi, orbit_pass)

# Print the SAR image collection.
print('SAR Image Collection:', sar_image_coll.getInfo())

SAR Image Collection: {'type': 'ImageCollection', 'bands': [], 'version': 1742903248015429, 'id': 'COPERNICUS/S1_GRD_FLOAT', 'features': [{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [27870, 21698], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 500169.82590810367, 0, -10, 1547022.4872775946]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [27870, 21698], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 500169.82590810367, 0, -10, 1547022.4872775946]}], 'version': 1718270110464424, 'id': 'COPERNICUS/S1_GRD_FLOAT/S1A_IW_GRDH_1SDV_20220310T224502_20220310T224527_042265_05099B_6810', 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.3', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_facility_country': 'France', 'GRD_Post_Processing_facility_org': 'ESA', 'transmitterReceiverPolarisation': ['VV', 'VH'], 'GRD_Post_Processing_start': 1647409057187, 'sliceNumber': 4, 'GRD

In [ ]:

# Function to get dates in 'YYYY-MM-dd' format.
def get_dates(dd):
    return ee.Date(dd).format('YYYY-MM-dd')

# Function to get a SAR Mosaic clipped to the study area.
def mosaic_sar(dates1):
    dates1 = ee.Date(dates1)
    image_filt = sar_image_coll \
        .filterDate(dates1, dates1.advance(1, 'day'))
    return image_filt.mosaic() \
        .clip(aoi) \
        .set({
            'system:time_start': dates1.millis(),
            'dateYMD': dates1.format('YYYY-MM-dd')
        })

# Function to get a SAR Collection of mosaics by date.
dates_mosaic = ee.List(sar_image_coll.aggregate_array('system:time_start')) \
    .map(get_dates) \
    .distinct()

# Get a SAR List and Image Collection of mosaics by date.
get_mosaic_list = dates_mosaic.map(mosaic_sar)
get_mosaic_coll = ee.ImageCollection(get_mosaic_list)

# Print the mosaic collection.
print('get Mosaic Collection:', get_mosaic_coll.getInfo())


get Mosaic Collection: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1646870400000, 'dateYMD': '2022-03-10', 'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1647907200000, 'dateYMD': '2022-03-22', 'system:index': '1'}}, {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1,

In [ ]:
# Assuming 'get_mosaic_coll' is the Python variable for the ImageCollection
# Get a list of available dates in the ImageCollection
available_dates = get_mosaic_coll.aggregate_array('dateYMD').getInfo()
print(f"Available dates in ImageCollection: {available_dates}")

Available dates in ImageCollection: ['2022-03-06', '2022-03-11', '2022-03-18', '2022-03-23', '2022-03-30', '2022-04-04', '2022-04-11', '2022-04-16', '2022-04-23', '2022-04-28', '2022-05-05', '2022-05-10', '2022-05-17', '2022-05-22', '2022-05-29', '2022-06-03', '2022-06-10', '2022-06-15', '2022-06-22', '2022-06-27', '2022-07-04', '2022-07-09', '2022-07-16', '2022-07-21', '2022-07-28']


In [ ]:
# Select the first and last available dates for visualization
# Make sure the dates exist in the available_dates list
first_date = available_dates[0]
last_date = available_dates[-1]

print(f"First available date: {first_date}")
print(f"Last available date: {last_date}")

First available date: 2022-03-06
Last available date: 2022-07-28


In [ ]:

# Visualization parameters for the SAR image.
sar_vis = {
    'bands': ['VV', 'VH', 'VV'],
    'min': [-18, -23, 3],
    'max': [-4, -11, 15]
}

# Filter and display specific SAR images for given dates.
image1 = get_mosaic_coll \
    .filter(ee.Filter.eq('dateYMD', '2022-03-10')) \
    .first().log10().multiply(10.0)

image2 = get_mosaic_coll \
    .filter(ee.Filter.eq('dateYMD', '2022-07-20')) \
    .first().log10().multiply(10.0)


In [ ]:
# Add Map
Map = geemap.Map()
Map.centerObject(aoi, 11)
Map.setOptions('SATELLITE')
Map.addLayer(aoi_outline, {'palette': 'red'}, 'Area of Study')
Map.addLayer(image1, sar_vis, 'Sentinel-1 | 2022-03-06')
Map.addLayer(image2, sar_vis, 'Sentinel-1 | 2022-07-28')
Map

Map(center=[12.983981834348848, 105.80606449999864], controls=(WidgetControl(options=['position', 'transparent…